In [2]:
import pandas as pd
import numpy as np
import duckdb
%run Categorization.ipynb

# in comparison to today's stats, statistics may have vary due to the season being on-going
pd.set_option('display.max_columns', None)


regular_dataset_2023 = pd.read_csv('datasets/regular/regular_dataset_2023_2024.csv')
advanced_dataset_2023 = pd.read_csv('datasets/advanced/advanced_dataset_2023_2024.csv')
shooting_splits_2023 = pd.read_csv('datasets//shootings_splits/SP_2023-2024.csv')
shooting_splits_2023

FileNotFoundError: [Errno 2] No such file or directory: 'datasets//shootings_splits/SP_2023-2024.csv'

In [ ]:
# In the case of multiples occurrence of a player, we only keep
regular_dataset_2023 = regular_dataset_2023[~regular_dataset_2023['Rk'].duplicated(keep='first')]
advanced_dataset_2023 = advanced_dataset_2023[~advanced_dataset_2023['Rk'].duplicated(keep='first')]

In [ ]:
regular_dataset_2023

In [ ]:
advanced_dataset_2023

In [ ]:
regular_dataset_2023.columns

In [ ]:
advanced_dataset_2023.columns

In [4]:
# concatenate the two tables, erasing the reccurent columns
start_columns = advanced_dataset_2023.columns.get_loc('PER')
columns_to_concat = advanced_dataset_2023.columns[start_columns:]
columns_to_concat
full_dataset_2024_raw = pd.concat([regular_dataset_2023, advanced_dataset_2023[columns_to_concat]], axis=1)
# clear the dataframe from duplicated columns
duplicated_columns = full_dataset_2024_raw.columns[full_dataset_2024_raw.columns.duplicated()]
full_dataset_2024 = full_dataset_2024_raw.drop(columns=duplicated_columns)
full_dataset_2024 = full_dataset_2024.drop(columns=['Unnamed: 24', 'Unnamed: 19'])
full_dataset_2024.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP'],
      dtype='object')

In [5]:
def get_glossary(glossary_file, columns):
    with open(glossary_file, 'r') as file:
        glossary_lines = file.readlines()
        
    filtered_lines = []

    for line in glossary_lines:
        if any(line.startswith(column) for column in columns):
            filtered_lines.append(line.strip() + '\n')  # Add newline character
        else:
            filtered_lines.append(line.strip() + '\n')  # Add newline character for all lines
        
    return filtered_lines

In [6]:
def add_return_to_lines(glossary_file):
    # Read the contents of the file
    with open(glossary_file, 'r') as file:
        lines = file.readlines()

    # Modify each line to add a return at the end
    modified_lines = [line.strip() + '\n' for line in lines]

    # Write the modified contents back to the file
    with open(glossary_file, 'w') as file:
        file.writelines(modified_lines)

In [7]:
filtered_glossary = get_glossary('glossary.txt', full_dataset_2024.columns)
filtered_glossary

['Rk -- Rank\n',
 'Pos -- Position\n',
 "Age -- Player's age on February 1 of the season\n",
 'Tm -- Team\n',
 'G -- Games\n',
 'MP -- Minutes Played\n',
 'PER -- Player Efficiency Rating\n',
 'A measure of per-minute production standardized such that the league average is 15.\n',
 'TS% -- True Shooting Percentage\n',
 'A measure of shooting efficiency that takes into account 2-point field goals, 3-point field goals, and free throws.\n',
 '3PAr -- 3-Point Attempt Rate\n',
 'Percentage of FG Attempts from 3-Point Range\n',
 'FTr -- Free Throw Attempt Rate\n',
 'Number of FT Attempts Per FG Attempt\n',
 'ORB% -- Offensive Rebound Percentage\n',
 'An estimate of the percentage of available offensive rebounds a player grabbed while they were on the floor.\n',
 'DRB% -- Defensive Rebound Percentage\n',
 'An estimate of the percentage of available defensive rebounds a player grabbed while they were on the floor.\n',
 'TRB% -- Total Rebound Percentage\n',
 'An estimate of the percentage of av

In [8]:
def write_filtered_glossary_to_file(filtered_glossary, output_file):
    with open(output_file, 'w') as file:
        for line in filtered_glossary:
            file.write(line)

write_filtered_glossary_to_file(filtered_glossary, 'filtered_glossary.txt')

In [9]:
# Remove players that played less than 24MPG so far, and less than 25 games
full_dataset_2024 = full_dataset_2024[(full_dataset_2024['MP'] >= 24) & (full_dataset_2024['G'] >= 25)]

# Round numbers
full_dataset_2024.iloc[:, 5:] = full_dataset_2024.iloc[:, 5:].round(2)

In [10]:
full_dataset_2024 = full_dataset_2024.reset_index()
full_dataset_2024

,index,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,3,2,Bam Adebayo,C,26,MIA,42,42,34.5,7.6,15.0,0.51,0.0,0.3,0.09,7.6,14.7,0.52,0.51,5.0,6.4,0.77,2.2,8.3,10.5,4.0,1.0,1.0,2.4,2.5,20.2,19.3,0.57,0.02,0.43,7.1,27.7,17.2,19.4,1.4,2.8,12.0,25.9,1.4,2.4,3.8,0.13,0.3,1.3,1.6,1.3
1,5,4,Santi Aldama,PF,23,MEM,40,14,24.4,4.0,9.2,0.44,1.7,4.8,0.36,2.3,4.4,0.52,0.53,0.9,1.4,0.62,1.2,4.5,5.7,2.0,0.7,0.7,1.2,1.5,10.7,13.2,0.54,0.52,0.15,5.1,21.0,12.7,12.5,1.3,2.6,10.5,19.4,0.0,1.3,1.3,0.06,-0.6,0.4,-0.2,0.5
2,7,6,Grayson Allen,SG,28,PHO,46,46,32.9,4.3,8.3,0.52,2.6,5.2,0.49,1.7,3.2,0.55,0.67,1.7,1.9,0.88,0.7,3.3,4.0,3.1,0.9,0.6,1.4,2.2,12.8,13.7,0.70,0.62,0.22,2.4,11.0,7.0,12.6,1.3,1.7,13.0,13.9,2.8,1.3,4.1,0.13,1.0,0.0,1.0,1.1
3,8,7,Jarrett Allen,C,25,CLE,45,45,30.4,6.3,9.8,0.64,0.0,0.1,0.00,6.3,9.8,0.64,0.64,2.8,3.7,0.76,3.5,7.0,10.5,2.8,0.8,1.2,1.8,2.3,15.4,22.0,0.67,0.01,0.38,13.0,24.9,19.1,13.4,1.3,3.5,13.5,18.9,3.8,2.7,6.4,0.22,1.8,1.7,3.5,1.9
4,11,10,Giannis Antetokounmpo,PF,29,MIL,50,50,35.3,11.6,19.1,0.61,0.5,1.8,0.25,11.2,17.2,0.65,0.62,7.4,11.2,0.66,2.6,8.6,11.2,6.4,1.4,1.1,3.6,3.0,31.1,29.2,0.65,0.10,0.59,8.3,26.0,17.4,30.6,1.8,2.5,13.0,33.2,6.0,2.4,8.4,0.23,6.2,2.0,8.1,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,568,523,Patrick Williams,PF,22,CHI,43,30,27.3,3.7,8.3,0.44,1.4,3.4,0.40,2.3,4.9,0.47,0.52,1.2,1.5,0.79,1.1,2.8,3.9,1.5,0.9,0.8,1.3,2.0,10.0,10.9,0.55,0.41,0.18,4.3,11.7,7.9,7.8,1.7,2.9,12.8,16.6,0.1,1.2,1.3,0.05,-2.4,0.0,-2.3,-0.1
183,570,525,Vince Williams Jr.,SG,23,MEM,40,21,25.7,2.9,6.7,0.44,1.6,4.1,0.38,1.4,2.6,0.53,0.56,1.8,2.2,0.80,1.3,4.3,5.6,2.4,0.8,0.6,1.3,2.5,9.2,12.8,0.60,0.62,0.34,5.2,19.1,11.9,13.7,1.5,2.2,14.3,14.9,1.0,1.3,2.3,0.11,-1.6,1.0,-0.6,0.4
184,572,527,Zion Williamson,PF,23,NOP,41,41,30.3,8.7,14.8,0.59,0.1,0.3,0.36,8.6,14.6,0.59,0.59,4.6,6.9,0.68,1.7,3.7,5.4,4.8,1.0,0.4,2.5,2.4,22.1,22.5,0.62,0.02,0.46,6.3,13.5,9.9,25.9,1.5,1.4,12.3,28.8,2.8,1.4,4.2,0.16,3.4,0.3,3.7,1.8
185,581,534,Trae Young,PG,25,ATL,46,46,36.6,8.5,19.7,0.43,3.4,8.9,0.38,5.1,10.8,0.48,0.52,6.7,7.8,0.86,0.5,2.3,2.8,10.9,1.5,0.2,4.3,2.0,27.1,20.7,0.59,0.45,0.40,1.3,7.1,4.1,44.1,1.9,0.5,15.8,30.7,3.7,0.4,4.2,0.12,5.3,-2.1,3.2,2.2


- Multiples entries for a single player --> Happens when a player has been traded, line with ['Tm'] = TOT keeped among all lines
- Removed player MPG < 24 and G < 25
- Concatenated standard stats with advanced stats 

In [11]:
#Extract and tranform shootings %FGA to cocnat with the full dataset

shooting_splits_2023 = pd.read_csv('shooting_splits_2023.csv')
shooting_splits_2023.columns = shooting_splits_2023.iloc[0]
shooting_splits_2023 = shooting_splits_2023.drop(shooting_splits_2023.index[0])
shooting_splits_2023 = shooting_splits_2023[(shooting_splits_2023['MP'].astype(int) / shooting_splits_2023['G'].astype(int) >= 24) & (shooting_splits_2023['G'].astype(int) >= 25)].reset_index()
shooting_splits_2023

FileNotFoundError: [Errno 2] No such file or directory: 'shooting_splits_2023.csv'

In [3]:


full_dataset_2024v2 = pd.merge(full_dataset_2024, shooting_splits_2023, on ='Player', how='left').reset_index()

stats_to_keep = ['Pos_x', '3P%_x', 'Tm']

cols_to_drop = [col for col in full_dataset_2024v2 if col not in stats_to_keep and isinstance(col, str) and (col.endswith('_x') or col.endswith('_y'))]

full_dataset_2024v2 = full_dataset_2024v2.drop(columns=cols_to_drop)
cols_to_keep = []

for col in full_dataset_2024v2.columns:
    if col not in cols_to_keep:
        cols_to_keep.append(col)
        
full_dataset_2024v2 = full_dataset_2024v2[cols_to_keep].dropna(axis=1, how='all')
full_dataset_2024v2.rename(columns={'3P%_x' : '3P%', 'Dist.' : 'Average distance (ft.) of FGA', '-9999' : 'player_code', '0-3' : '%FGA 0-3', '3-10' : '%FGA 3-10', '10-16' : '%FGA 10-16', '16-3P' : '%FGA 16-3P', 'Pos_x' : 'Pos'}, inplace=True)
full_dataset_2024v2.drop(columns=['#', '%FGA', '%3PA', 'Att.', 'player_code'],inplace=True)
full_dataset_2024v2 = full_dataset_2024v2.loc[:, ~full_dataset_2024v2.columns.duplicated()]
full_dataset_2024v2.iloc[:, -4:] = full_dataset_2024v2.iloc[:, -4:].apply(lambda x : x.astype(float) * 100)
full_dataset_2024v2['%FGA 3P'] = 100 - full_dataset_2024v2.iloc[:, -4:].sum(axis=1)
full_dataset_2024v2

NameError: name 'full_dataset_2024' is not defined

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Player'] == 'Kevin Durant']

In [ ]:
# Get the mean of every value for analysis needs

# Select only the columns that are numeric for mean calculation
numeric_columns = full_dataset_2024v2.iloc[:, :].select_dtypes(include='number')

# Calculate the mean
avg = numeric_columns.mean().round(1)

# Assign the calculated mean to a new row called 'Average'
full_dataset_2024v2.loc['Average'] = avg

# Display the updated DataFrame
full_dataset_2024v2

In [ ]:
# check if the mean is correct with an other sample
full_dataset_2024v2.insert(21, 'AST/TOV', (full_dataset_2024v2['AST']/full_dataset_2024v2['TOV']))
full_dataset_2024v2['AST/TOV'] = round(full_dataset_2024v2['AST/TOV'], 1)
full_dataset_2024v2

In [ ]:
# if std_areas_FGA is low -> shots are more likely to be evenly splitted into the 5 distances than is the std_areas_FGA is high

fga_area_columns = full_dataset_2024v2.iloc[:, -5:]
full_dataset_2024v2['std_areas_FGA'] = fga_area_columns.apply(lambda row:np.std(row), axis=1)
full_dataset_2024v2['std_areas_FGA'] = round(full_dataset_2024v2['std_areas_FGA'], 1)
full_dataset_2024v2

In [ ]:
# Create a copy of the original DataFrame
full_dataset_ranked_2024v2 = full_dataset_2024v2.copy()

# Select columns from the 4th onwards
columns_to_rank = full_dataset_ranked_2024v2.columns[3:]

# Iterate over the selected columns and calculate the ranked values
for column in columns_to_rank:
    # Convert column values to numeric, ignoring errors
    column_data = pd.to_numeric(full_dataset_ranked_2024v2[column], errors='coerce')
    
    # Replace missing or non-numeric values with NaN
    column_data = column_data.fillna(0)
    
    # Calculate the percentile ranks for the column and reverse the ranking order
    percentile_ranks = 1 - column_data.rank(pct=True)
    
    # Calculate the range numbers based on the reversed percentile ranks
    range_numbers = (percentile_ranks * 10).astype(int) + 1
    
    # Create a new column with the ranked values next to the original column
    new_column_name = f'{column} ranked'
    full_dataset_ranked_2024v2.insert(full_dataset_ranked_2024v2.columns.get_loc(column) + 1, new_column_name, range_numbers)

# Display the updated dataset
full_dataset_ranked_2024v2

In [ ]:
full_dataset_ranked_2024v2[full_dataset_ranked_2024v2['USG% ranked'] == 1].sort_values(by=['USG%'], ascending=False)

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['%FGA 0-3'] > 50]

In [ ]:
full_dataset_2024v2.insert(3, 'Offensive Profile', 0)
full_dataset_2024v2.insert(4, 'Defensive Profile', 0)
# Round numbers
full_dataset_2024.iloc[:, 5:] = full_dataset_2024.iloc[:, 5:].round(2)
full_dataset_2024v2

In [ ]:
# Iterate over each row in the DataFrame
for index, row in full_dataset_2024v2.iterrows():
    # Call offensive_profile function and assign the result to 'Offensive Profile' column
    full_dataset_2024v2.at[index, 'Offensive Profile'] = offensive_profile(row, full_dataset_ranked_2024v2.loc[index])
    # Call defensive_profile function and assign the result to 'Defensive Profile' column
    full_dataset_2024v2.at[index, 'Defensive Profile'] = defensive_profile(row, full_dataset_ranked_2024v2.loc[index])

In [ ]:
full_dataset_2024v2

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'Creator/Facilitator']

In [ ]:
full_dataset_ranked_2024v2[full_dataset_ranked_2024v2['Player'] == 'Klay Thompson']

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'Pure Shooter/Stretcher']

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'Paint Threat']

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'Slasher']

Paint Threat et Slasher en concurrence 

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'Versatile Scorer']

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Offensive Profile'] == 'No significant offensive role']

In [ ]:
full_dataset_2024v2[full_dataset_2024v2['Player'] == 'Klay Thompson']

In [ ]:
full_dataset_2024v2.to_csv('full_dataset_2024v2.csv')
full_dataset_ranked_2024v2.to_csv('full_dataset_ranked_2024v2.csv')


In [ ]:
ranking_east_2024 = pd.read_csv("ranking_east_2023.csv")
ranking_west_2024 = pd.read_csv("ranking_west_2023.csv")

In [ ]:
ranking_west_2024